In [1]:
#Dependencies
import pandas as pd
import numpy as np

In [2]:
# Use Pandas to read your cleaned measurements and stations CSV data.
clean_hawaii_measurements_df = pd.read_csv('clean_hawaii_measurements.csv')
clean_hawaii_stations_df = pd.read_csv('clean_hawaii_stations.csv')

In [3]:
clean_hawaii_measurements_df.head(5)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.16,73


In [4]:
clean_hawaii_stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [5]:
## Use the `engine` and connection string to create a database called `hawaii.sqlite`.

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [6]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [7]:
# Use `declarative_base` and create ORM classes for each table.

# Create the ORM class for measurements
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurements'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [8]:
# Create the ORM class for stations
class Station(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [9]:
# create the tables in the database using `create_all`.
Base.metadata.create_all(engine)

In [10]:
# Check if the tables were created in the database
import sqlite3

db_filename = 'hawaii.sqlite'
db=sqlite3.connect(db_filename)
cur = db.cursor()
result = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print(result)


[('stations',), ('measurements',)]


In [11]:
# define a function to insert the dataframes into the sqlite tables
def insert_data(df, table_name):

    # Use Orient='records' to create a list of data to write
    data = df.to_dict(orient='records')
    
    # Use `table.delete()` to remove any pre-existing data.
    conn.execute(table_name.delete())

    # Use `table.insert()` to insert the data into the table
    # The SQL table is populated during this step
    conn.execute(table_name.insert(), data)

# Call the function for inserting data for each table
insert_data(clean_hawaii_measurements_df, Measurement.__table__)
insert_data(clean_hawaii_stations_df, Station.__table__)

In [12]:
# Check if the data was inserted correctly for the table `measurement`
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.16, 73.0)]

In [13]:
# Check if the data was inserted correctly for the table `station`
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]